In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [40]:
import pandas as pd
from src.inference import fetch_next_hour_predictions, load_batch_of_features_from_store
from src.plot_utils import plot_aggregated_time_series

current_date = pd.Timestamp.now(tz="Etc/UTC")

features = load_batch_of_features_from_store(current_date)

2025-03-05 17:22:04,176 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 17:22:04,178 INFO: Initializing external client
2025-03-05 17:22:04,178 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-03-05 17:22:04,844 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214671
Fetching data from 2025-02-04 22:22:04.176614+00:00 to 2025-03-05 21:22:04.176614+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.10s) 


In [41]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-04 23:00:00
1,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,0,3,2025-03-04 23:00:00
2,0,2,0,3,0,0,0,0,4,6,...,6,2,1,5,3,3,5,3,4,2025-03-04 23:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-04 23:00:00
4,2,1,1,0,0,0,1,2,6,4,...,5,0,3,1,2,0,2,0,7,2025-03-04 23:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,0,0,0,0,0,0,2,1,1,...,0,0,0,0,0,0,0,0,259,2025-03-04 23:00:00
248,0,1,0,0,0,0,0,0,1,4,...,1,0,2,1,1,0,1,0,260,2025-03-04 23:00:00
249,9,6,4,0,1,0,1,7,8,15,...,26,28,58,46,34,25,23,22,261,2025-03-04 23:00:00
250,6,2,1,0,5,6,13,66,151,147,...,140,111,101,87,48,30,25,10,262,2025-03-04 23:00:00


In [42]:
predictions = fetch_next_hour_predictions()

2025-03-05 17:22:15,010 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 17:22:15,013 INFO: Initializing external client
2025-03-05 17:22:15,013 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-03-05 17:22:15,588 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214671
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.41s) 
Current UTC time: 2025-03-05 22:22:15.010842+00:00
Next hour: 2025-03-05 23:00:00+00:00
Found 0 records


In [43]:
predictions


,pickup_location_id,predicted_demand,pickup_hour


In [44]:
top10 = (
    predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].to_list()
)

In [45]:
top10

[]

In [46]:
top10

[]

In [47]:
features["pickup_location_id"] == 22

0      False
1      False
2      False
3      False
4      False
       ...  
247    False
248    False
249    False
250    False
251    False
Name: pickup_location_id, Length: 252, dtype: bool

In [48]:
features_cp = features.copy()

In [49]:
from datetime import timedelta

# Get your data
features = features_cp[features_cp["pickup_location_id"] == 161]
prediction = predictions[predictions["pickup_location_id"] == 161]

time_series_columns = [
    col for col in features.columns if col.startswith("rides_t-")
]

# Get time series values
time_series_values = [features[col].iloc[0] for col in time_series_columns]

# Handle empty prediction case
if len(prediction["predicted_demand"]) == 0:
    prediction_values = []
else:
    prediction_values = prediction["predicted_demand"].tolist()

# Combine values
combined_values = time_series_values + prediction_values

# Convert pickup_hour Series to single timestamp
pickup_hour = pd.Timestamp(features["pickup_hour"].iloc[0])

# Create date range with exact number of periods
time_series_dates = pd.date_range(
    end=pickup_hour,
    periods=len(combined_values),  # Use exact number of periods
    freq="h",
)

# Verify lengths match
print(f"Length of time_series_dates: {len(time_series_dates)}")
print(f"Length of combined values: {len(combined_values)}")

# Create DataFrame
historical_df = pd.DataFrame(
    {"datetime": time_series_dates, "rides": combined_values}
)
# from datetime import timedelta
# features = features_cp[features_cp["pickup_location_id"] == 161]
# prediction = predictions[predictions["pickup_location_id"] == 161]
# time_series_columns = [
#     col for col in features.columns if col.startswith("rides_t-")
# ]
# time_series_values = [features[col].iloc[0] for col in time_series_columns] + prediction["predicted_demand"].to_list()

# # Convert pickup_hour Series to single timestamp
# pickup_hour = pd.Timestamp(features["pickup_hour"].iloc[0])

# # Generate corresponding timestamps for the time series
# time_series_dates = pd.date_range(
#     start=pickup_hour - timedelta(hours=len(time_series_columns)),
#     end=pickup_hour,
#     freq="h",
# )

# # Create a DataFrame for the historical data
# historical_df = pd.DataFrame(
#     {"datetime": time_series_dates, "rides": time_series_values}
# )

Length of time_series_dates: 672
Length of combined values: 672


In [50]:
from datetime import timedelta

# Check lengths of data
features = features_cp[features_cp["pickup_location_id"] == 161]
prediction = predictions[predictions["pickup_location_id"] == 161]

time_series_columns = [
    col for col in features.columns if col.startswith("rides_t-")
]

# Print lengths to debug
print(f"Number of time series columns: {len(time_series_columns)}")
print(f"Number of prediction values: {len(prediction['predicted_demand'])}")
print(f"Total expected length: {len(time_series_columns) + len(prediction['predicted_demand'])}")

# Fix: Make sure time_series_values length matches time_series_dates length
time_series_values = [features[col].iloc[0] for col in time_series_columns]
prediction_values = prediction["predicted_demand"].to_list()

# Convert pickup_hour Series to single timestamp
pickup_hour = pd.Timestamp(features["pickup_hour"].iloc[0])

# Generate corresponding timestamps for the time series
time_series_dates = pd.date_range(
    start=pickup_hour - timedelta(hours=len(time_series_columns)),
    end=pickup_hour,
    freq="h",
)

# Print lengths again to confirm
print(f"Length of time_series_dates: {len(time_series_dates)}")
print(f"Length of time_series_values + prediction: {len(time_series_values) + len(prediction_values)}")

# Make sure the lengths match before creating DataFrame
if len(time_series_dates) == len(time_series_values) + len(prediction_values):
    combined_values = time_series_values + prediction_values
else:
    # Adjust time_series_dates to match the combined length
    time_series_dates = pd.date_range(
        end=pickup_hour,
        periods=len(time_series_values) + len(prediction_values),
        freq="h",
    )
    combined_values = time_series_values + prediction_values

# Create a DataFrame for the historical data
historical_df = pd.DataFrame(
    {"datetime": time_series_dates, "rides": combined_values}
)

Number of time series columns: 672
Number of prediction values: 0
Total expected length: 672
Length of time_series_dates: 673
Length of time_series_values + prediction: 672


In [51]:
historical_df

,datetime,rides
0,2025-02-05 00:00:00,121
1,2025-02-05 01:00:00,64
2,2025-02-05 02:00:00,14
3,2025-02-05 03:00:00,5
4,2025-02-05 04:00:00,5
...,...,...
667,2025-03-04 19:00:00,657
668,2025-03-04 20:00:00,521
669,2025-03-04 21:00:00,550
670,2025-03-04 22:00:00,526
